## 22. 시계열 데이터4
### 주어진 데이터(basic2.csv)에서 주 단위 Sales의 합계를 구하고, 가장 큰 값을 가진 주와 작은 값을 가진 주의 차이를 구하시오(절대값)
- 데이터셋 : basic2.csv

In [1]:
import pandas as pd

df = pd.read_csv('../data/basic2.csv')

In [2]:
df['Date'] = pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    730 non-null    datetime64[ns]
 1   Sales   730 non-null    int64         
 2   PV      730 non-null    int64         
 3   UV      684 non-null    float64       
 4   Events  730 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 28.6 KB


In [3]:
# 주 단위 Sales의 합계
df = df.set_index('Date')          # Date 컬럼을 인덱스로 설정...

df = df.resample('W').sum()

In [4]:
df.head()

,Sales,PV,UV,Events
Date,,,,
2022-01-02,22748304,401318,6710.0,1
2022-01-09,33373822,1148464,15189.0,2
2022-01-16,20754519,283773,9344.0,0
2022-01-23,21880751,538633,9841.0,1
2022-01-30,28878452,560814,7808.0,1


In [5]:
# 합계가 가장 큰 값을 가진 주와 작은 값을 가진 주의 차이
df['Sales'].max() - df['Sales'].min()

91639050

<br><br>

## 23. 중복 데이터 제거
### f1의 결측치를 채운 후 age 컬럼의 중복 제거 전과 후의 'f1' 중앙값 차이를 구하시오
### - 결측치는 'f1' 데이터에서 큰 값 순으로 정렬했을 때 10번째에 위치한 값으로 채운다.
### - 중복 데이터 발생시 뒤에 나오는 데이터를 삭제함
### - 최종 결과값은 절대값으로 출력
(중복 제거 기준: 제공된 데이터 순서대로 중복 데이터 발생시 뒤에 나오는 데이터를 삭제함)

- 데이터셋 : basic1.csv

In [6]:
import pandas as pd

df = pd.read_csv('../data/basic1.csv')
df.head()

,id,age,city,f1,f2,f3,f4,f5
0,id01,2.0,서울,NaN,0,NaN,ENFJ,91.297791
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826
2,id03,27.0,서울,61.0,1,NaN,ISTJ,17.252986
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869


In [11]:
# f1의 결측치 10번째로 큰 값으로 대체

df.sort_values('f1', ascending=False)[:10]

,id,age,city,f1,f2,f3,f4,f5
56,id57,3.0,대구,111.0,0,NaN,ISFJ,29.269869
7,id08,38.0,서울,101.0,1,NaN,INFJ,83.685380
57,id58,0.0,대구,100.0,2,NaN,ESTP,33.308999
73,id74,45.0,경기,98.0,0,NaN,ESTP,52.667078
71,id72,8.0,경기,97.0,0,NaN,ESTJ,97.381034
69,id70,-9.0,경기,96.0,1,NaN,ISTP,48.431184
77,id78,92.0,경기,96.0,1,NaN,INTJ,69.730313
32,id33,47.0,부산,94.0,0,NaN,ENFJ,17.252986
48,id49,75.0,대구,88.0,0,NaN,INTP,37.113739
44,id45,97.0,대구,88.0,0,NaN,ENFJ,13.049921


In [12]:
df['f1'] = df['f1'].fillna(88.0)

In [15]:
# age 컬럼의 중복 제거 전과 후의 'f1' 중앙값 차이
drop_dup = df.copy()

drop_dup = drop_dup.drop_duplicates('age', keep='first')        # keep: 중복값이 여러개일 때 남길 값 선택

In [16]:
median1 = df['f1'].median()
median2 = drop_dup['f1'].median()

print(abs(median1-median2))

0.5


<br><br>

## 24. 시계열 데이터5
### 주어진 데이터(basic2.csv)에서 "pv"컬럼으로 1일 시차(lag)가 있는 새로운 컬럼을 만들고(예: 1월 2일에는 1월 1일 pv데이터를 넣고, 1월 3일에는 1월 2일 pv데이터를 넣음),새로운 컬럼의 1월 1일은 다음날(1월 2일)데이터로 결측치를 채운 다음, Events가 1이면서 Sales가 1000000이하인 조건에 맞는 새로운 컬럼 합을 구하시오
- 데이터셋 : basic2.csv

In [1]:
import pandas as pd

df = pd.read_csv('../data/basic2.csv')
df.head()

,Date,Sales,PV,UV,Events
0,2022-01-01,22711525,397349,4421.0,1
1,2022-01-02,36779,3969,2289.0,0
2,2022-01-03,13943875,373890,NaN,1
3,2022-01-04,8088593,142054,2157.0,0
4,2022-01-05,2395374,190684,NaN,0


In [7]:
# pv와 1일 시차가 있는 새로운 컬럼 만들기
### df.shift(시차 주기)
df['prev_pv'] = df['PV'].shift(1)
df.head()

,Date,Sales,PV,UV,Events,prev_pv
0,2022-01-01,22711525,397349,4421.0,1,NaN
1,2022-01-02,36779,3969,2289.0,0,397349.0
2,2022-01-03,13943875,373890,NaN,1,3969.0
3,2022-01-04,8088593,142054,2157.0,0,373890.0
4,2022-01-05,2395374,190684,NaN,0,142054.0


In [9]:
# 새로운 컬럼의 1월 1일은 다음날(1월 2일)데이터로 결측치를 채움

print(df['prev_pv'].isnull().sum())

df['prev_pv'] = df['prev_pv'].fillna(method='bfill')      # bfill: 뒤의 데이터로 채움, ffill: 앞의 데이터로 채움
df.head()

1


,Date,Sales,PV,UV,Events,prev_pv
0,2022-01-01,22711525,397349,4421.0,1,397349.0
1,2022-01-02,36779,3969,2289.0,0,397349.0
2,2022-01-03,13943875,373890,NaN,1,3969.0
3,2022-01-04,8088593,142054,2157.0,0,373890.0
4,2022-01-05,2395374,190684,NaN,0,142054.0


In [13]:
# Events가 1이면서 Sales가 1000000이하인 조건에 맞는 합
df[(df['Events']==1) & (df['Sales'] <= 1000000)]['prev_pv'].sum()

1894876.0